In [1]:
!pip install --upgrade scikit-learn

In [2]:
import sklearn

print("scikit-learn version:", sklearn.__version__)

scikit-learn version: 1.3.0


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d ebrahimhaquebhatti/pakistan-house-price-prediction

pakistan-house-price-prediction.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/pakistan-house-price-prediction.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [7]:
data=pd.read_csv("Entities.csv")

In [8]:
data.isnull().sum()

Unnamed: 0           0
property_id          0
location_id          0
page_url             0
property_type        0
price                0
location             0
city                 0
province_name        0
latitude             0
longitude            0
baths                0
purpose              0
bedrooms             0
date_added           0
agency           44071
agent            44072
Total_Area           0
dtype: int64

In [9]:
data.shape

(168446, 18)

In [10]:
data.head()

,Unnamed: 0,property_id,location_id,page_url,property_type,price,location,city,province_name,latitude,longitude,baths,purpose,bedrooms,date_added,agency,agent,Total_Area
0,0,237062,3325,https://www.zameen.com/Property/g_10_g_10_2_gr...,Flat,10000000,G-10,Islamabad,Islamabad Capital,33.679890,73.012640,2,For Sale,2,2/4/2019,NaN,NaN,1089.004
1,1,346905,3236,https://www.zameen.com/Property/e_11_2_service...,Flat,6900000,E-11,Islamabad,Islamabad Capital,33.700993,72.971492,3,For Sale,3,5/4/2019,NaN,NaN,15246.056
2,2,386513,764,https://www.zameen.com/Property/islamabad_g_15...,House,16500000,G-15,Islamabad,Islamabad Capital,33.631486,72.926559,6,For Sale,5,7/17/2019,NaN,NaN,2178.008
3,3,656161,340,https://www.zameen.com/Property/islamabad_bani...,House,43500000,Bani Gala,Islamabad,Islamabad Capital,33.707573,73.151199,4,For Sale,4,4/5/2019,NaN,NaN,10890.000
4,4,841645,3226,https://www.zameen.com/Property/dha_valley_dha...,House,7000000,DHA Defence,Islamabad,Islamabad Capital,33.492591,73.301339,3,For Sale,3,7/10/2019,Easy Property,Muhammad Junaid Ceo Muhammad Shahid Director,2178.008


In [11]:
columns_to_drop = ['Unnamed: 0','property_id','location_id','page_url','date_added','agent','agency','latitude','longitude','province_name']
data = data.drop(columns=columns_to_drop)

In [12]:
data.head()

,property_type,price,location,city,baths,purpose,bedrooms,Total_Area
0,Flat,10000000,G-10,Islamabad,2,For Sale,2,1089.004
1,Flat,6900000,E-11,Islamabad,3,For Sale,3,15246.056
2,House,16500000,G-15,Islamabad,6,For Sale,5,2178.008
3,House,43500000,Bani Gala,Islamabad,4,For Sale,4,10890.000
4,House,7000000,DHA Defence,Islamabad,3,For Sale,3,2178.008


In [13]:
data.describe()

,price,baths,bedrooms,Total_Area
count,1.684460e+05,168446.000000,168446.000000,1.684460e+05
mean,1.776576e+07,2.874227,3.179422,1.394239e+04
std,3.531003e+07,2.463400,1.971401,8.623647e+05
min,0.000000e+00,0.000000,0.000000,0.000000e+00
25%,1.750000e+05,0.000000,2.000000,1.905757e+03
50%,8.500000e+06,3.000000,3.000000,4.356016e+03
75%,1.950000e+07,4.000000,4.000000,1.197904e+04
max,2.000000e+09,403.000000,68.000000,3.387988e+08


In [14]:
data[data.baths==0].shape

(42382, 8)

In [15]:
data['marla']=data['Total_Area']/272.251

In [16]:
data['marla']=data['marla'].astype(int)
data['Total_Area']=data['Total_Area'].astype(int)

In [17]:
df2 = data[data['marla'] > 300].index
data.drop(df2, inplace=True)
data.reset_index(drop=True, inplace=True)  # Reset index and drop the old index column
print(data.shape)

(166761, 9)


In [18]:
data.tail()

,property_type,price,location,city,baths,purpose,bedrooms,Total_Area,marla
166756,House,26500000,Gadap Town,Karachi,0,For Sale,6,26136,96
166757,House,12500000,Gadap Town,Karachi,0,For Sale,3,2178,8
166758,House,27000000,Gadap Town,Karachi,0,For Sale,6,26136,96
166759,House,11000000,Gadap Town,Karachi,0,For Sale,3,21235,78
166760,House,9000000,Bahria Town Karachi,Karachi,3,For Sale,3,25591,94


In [19]:
df=data.loc[(data['bedrooms']==0) | (data['baths']==0)].index
df

Int64Index([    17,     42,     44,     49,     50,     53,     54,     76,
                77,     79,
            ...
            166709, 166710, 166711, 166717, 166720, 166755, 166756, 166757,
            166758, 166759],
           dtype='int64', length=42362)

In [20]:
# Calculate the mean of non-zero bedrooms
mean_bedrooms = data.loc[data["bedrooms"] > 0, "bedrooms"].mean().astype(int)
mean_bedrooms

3

In [21]:
# Replace 0 bedrooms with the mean
data.loc[data["bedrooms"] == 0, "bedrooms"] = mean_bedrooms

In [22]:
# Calculate the mean of non-zero bathrooms
mean_baths = data.loc[data["baths"] > 0, "baths"].mean().astype(int)
mean_baths

3

In [23]:
# Replace 0 bathrooms with the mean
data.loc[data["baths"] == 0, "baths"] = mean_baths

In [24]:
data.loc[17]

property_type                  Flat
price                       1750000
location         PWD Housing Scheme
city                      Islamabad
baths                             3
purpose                    For Sale
bedrooms                          3
Total_Area                     4083
marla                            15
Name: 17, dtype: object

In [25]:
df=data.loc[(data['bedrooms']==0) | (data['baths']==0)].index
df.shape

(0,)

In [26]:
data['price_per_sqfeet']=data['price']/data['Total_Area']

In [27]:
data.head()

,property_type,price,location,city,baths,purpose,bedrooms,Total_Area,marla,price_per_sqfeet
0,Flat,10000000,G-10,Islamabad,2,For Sale,2,1089,4,9182.736455
1,Flat,6900000,E-11,Islamabad,3,For Sale,3,15246,56,452.577725
2,House,16500000,G-15,Islamabad,6,For Sale,5,2178,8,7575.757576
3,House,43500000,Bani Gala,Islamabad,4,For Sale,4,10890,39,3994.490358
4,House,7000000,DHA Defence,Islamabad,3,For Sale,3,2178,8,3213.957759


In [28]:
data.describe()

,price,baths,bedrooms,Total_Area,marla,price_per_sqfeet
count,1.667610e+05,166761.000000,166761.000000,166761.000000,166761.000000,1.667610e+05
mean,1.696111e+07,3.614388,3.567675,9149.793717,33.431090,inf
std,3.156131e+07,1.837863,1.508731,12962.361140,47.513804,NaN
min,0.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000e+00
25%,1.690000e+05,3.000000,3.000000,1905.000000,7.000000,3.443526e+01
50%,8.500000e+06,3.000000,3.000000,4356.000000,16.000000,1.224365e+03
75%,1.900000e+07,4.000000,4.000000,11434.000000,42.000000,7.347539e+03
max,1.430000e+09,403.000000,28.000000,81675.000000,299.000000,inf


In [29]:
data['location'].value_counts()

DHA Defence                   21879
Bahria Town Karachi            8522
Bahria Town Rawalpindi         8467
Bahria Town                    6387
Gulistan-e-Jauhar              5851
                              ...  
Firdous Colony                    1
Samsani Road                      1
Gulfishan Town                    1
Steam Power Station Colony        1
Sundar                            1
Name: location, Length: 1500, dtype: int64

In [30]:
data['location']=data['location'].apply(lambda x: x.strip(' '))
location_counts=data['location'].value_counts()
location_counts

DHA Defence                   21879
Bahria Town Karachi            8522
Bahria Town Rawalpindi         8467
Bahria Town                    6387
Gulistan-e-Jauhar              5851
                              ...  
Firdous Colony                    1
Samsani Road                      1
Gulfishan Town                    1
Steam Power Station Colony        1
Sundar                            1
Name: location, Length: 1500, dtype: int64

In [31]:
location_less_than_10=location_counts[location_counts<=10]
location_less_than_10

IJP Road                      10
Mian Amiruddin Park           10
Zaman Colony                  10
Kaleem Shaheed Colony No 1    10
Shah Khalid Colony            10
                              ..
Firdous Colony                 1
Samsani Road                   1
Gulfishan Town                 1
Steam Power Station Colony     1
Sundar                         1
Name: location, Length: 821, dtype: int64

In [32]:
data['location']=data['location'].apply(lambda x: 'other' if x in location_less_than_10 else x)
data['location'].value_counts()

DHA Defence               21879
Bahria Town Karachi        8522
Bahria Town Rawalpindi     8467
Bahria Town                6387
Gulistan-e-Jauhar          5851
                          ...  
Shahrah-e-Pakistan           11
Rajgarh Road                 11
Model Colony                 11
Kashmir Colony               11
Aria Mohalla                 11
Name: location, Length: 680, dtype: int64

In [33]:
data.describe()

,price,baths,bedrooms,Total_Area,marla,price_per_sqfeet
count,1.667610e+05,166761.000000,166761.000000,166761.000000,166761.000000,1.667610e+05
mean,1.696111e+07,3.614388,3.567675,9149.793717,33.431090,inf
std,3.156131e+07,1.837863,1.508731,12962.361140,47.513804,NaN
min,0.000000e+00,1.000000,1.000000,0.000000,0.000000,0.000000e+00
25%,1.690000e+05,3.000000,3.000000,1905.000000,7.000000,3.443526e+01
50%,8.500000e+06,3.000000,3.000000,4356.000000,16.000000,1.224365e+03
75%,1.900000e+07,4.000000,4.000000,11434.000000,42.000000,7.347539e+03
max,1.430000e+09,403.000000,28.000000,81675.000000,299.000000,inf


In [34]:
(data['Total_Area']/data['bedrooms']).describe()

count    166761.000000
mean       2868.344056
std        3853.079696
min           0.000000
25%         544.400000
50%        1089.000000
75%        4356.000000
max       76230.000000
dtype: float64

In [35]:
data=data[((data['Total_Area']/data['bedrooms'])>=300)]
data.describe()

,price,baths,bedrooms,Total_Area,marla,price_per_sqfeet
count,1.561430e+05,156143.000000,156143.000000,156143.000000,156143.000000,156143.000000
mean,1.757474e+07,3.616422,3.541311,9707.351204,35.466899,3906.842148
std,3.244402e+07,1.566081,1.495527,13211.980470,48.434029,5195.163696
min,0.000000e+00,1.000000,1.000000,544.000000,2.000000,0.000000
25%,1.400000e+05,3.000000,3.000000,2178.000000,8.000000,28.655400
50%,9.000000e+06,3.000000,3.000000,5445.000000,19.000000,1011.994003
75%,2.000000e+07,4.000000,4.000000,11979.000000,44.000000,7116.620753
max,1.430000e+09,14.000000,28.000000,81675.000000,299.000000,316804.407713


In [36]:
data.shape

(156143, 10)

In [37]:
def  remove_outlier_sqft(df):
    df_output=pd.DataFrame()
    for key,subdf in data.groupby('location'):
        m=np.mean(subdf.price_per_sqfeet)
        st=np.std(subdf.price_per_sqfeet)
        gen_df=subdf[(subdf.price_per_sqfeet>(m-st))&(subdf.price_per_sqfeet <= (m+st))]
        df_output=pd.concat( [df_output , gen_df ], ignore_index=True)
    return df_output

data=remove_outlier_sqft(data)
data.describe()


,price,baths,bedrooms,Total_Area,marla,price_per_sqfeet
count,1.240610e+05,124061.000000,124061.000000,124061.000000,124061.000000,124061.000000
mean,1.359040e+07,3.542644,3.458839,10748.786452,39.288052,2753.022313
std,2.196683e+07,1.527906,1.459130,13969.293270,51.216867,3536.036595
min,0.000000e+00,1.000000,1.000000,544.000000,2.000000,0.000000
25%,8.500000e+04,3.000000,3.000000,2178.000000,8.000000,22.042616
50%,7.000000e+06,3.000000,3.000000,5445.000000,19.000000,720.253529
75%,1.700000e+07,4.000000,4.000000,13068.000000,48.000000,5693.296602
max,6.000000e+08,14.000000,28.000000,81675.000000,299.000000,38047.756494


In [38]:
df1=data.loc[(data['bedrooms']>=20) | (data['baths']>=10)].index
df1.shape

(496,)

In [39]:
data.shape

(124061, 10)

In [40]:
data.drop(df1, inplace=True)
data.reset_index(drop=True, inplace=True)  # Reset index and drop the old index column
print(data.shape)

(123565, 10)


In [41]:
data.describe()

,price,baths,bedrooms,Total_Area,marla,price_per_sqfeet
count,1.235650e+05,123565.000000,123565.000000,123565.000000,123565.000000,123565.000000
mean,1.350607e+07,3.516934,3.434290,10681.356436,39.042771,2756.381413
std,2.164201e+07,1.474673,1.398352,13869.246293,50.851673,3537.293736
min,0.000000e+00,1.000000,1.000000,544.000000,2.000000,0.000000
25%,8.500000e+04,3.000000,3.000000,2178.000000,8.000000,22.042616
50%,7.000000e+06,3.000000,3.000000,5445.000000,19.000000,721.500722
75%,1.680000e+07,4.000000,4.000000,13068.000000,48.000000,5693.296602
max,5.000000e+08,9.000000,16.000000,81675.000000,299.000000,38047.756494


In [42]:
status_counts = data['purpose'].value_counts()

print("Counts of properties for sale and for rent:")
print(status_counts)

Counts of properties for sale and for rent:
For Sale    81475
For Rent    42090
Name: purpose, dtype: int64


In [43]:
data.isnull().sum()

property_type       0
price               0
location            0
city                0
baths               0
purpose             0
bedrooms            0
Total_Area          0
marla               0
price_per_sqfeet    0
dtype: int64

In [44]:
data.shape

(123565, 10)

In [45]:
data.head()

,property_type,price,location,city,baths,purpose,bedrooms,Total_Area,marla,price_per_sqfeet
0,Flat,180000,7th Avenue,Islamabad,2,For Rent,3,13340,49,13.493253
1,Flat,90000,7th Avenue,Islamabad,2,For Rent,2,17424,64,5.165289
2,House,1000000,7th Avenue,Islamabad,8,For Rent,8,10890,39,91.827365
3,Room,5000,7th Avenue,Islamabad,3,For Rent,1,544,2,9.191176
4,Room,17000,7th Avenue,Islamabad,1,For Rent,2,1361,5,12.490816


In [46]:
data['Total_Area'].unique()

array([13340, 17424, 10890,   544,  1361,  2722,  1905,  3267, 18240,
       16335, 15518, 16607, 10617,  5445, 70785, 59895, 21780,  5989,
        1089,  1633, 14973, 34575, 11434, 12795,  2178,  6261, 18785,
       13884, 24230, 13068,   816,  5717, 23141,  9528, 12251,  3811,
        3539, 19329,  4900, 14429, 26136,  8712,  8984, 11979,  8439,
       10345,  9256,  9801, 15790,  4356, 23958,  6534,  7895,  7078,
       19602, 12523,  7623, 18513,  7350, 26680, 27497, 16879,  2450,
        2994,  4628, 54450, 28586,  6806, 20418,  4083, 35664, 10073,
       14157, 22869, 28314, 17968, 15246, 37570, 14701, 23413, 27769,
       25319, 31308, 34031, 21235, 17696, 31581, 41109, 22324, 17151,
       29130, 25047, 34848, 20691, 32670,  5172, 19874, 30219, 29403,
       48460, 50910, 29675, 32125, 33759, 38659, 44649, 42471, 11162,
       11706, 35937, 65340, 43560, 33486, 33214, 21507, 22596, 30492,
       37026, 81675, 36753, 25863, 76230, 31853, 25591, 46827, 32942,
       39476, 16062,

In [47]:
data['Total_Area'].dtype

dtype('int64')

In [48]:
data['property_type'].unique()

array(['Flat', 'House', 'Room', 'Upper Portion', 'Lower Portion',
       'Farm House', 'Penthouse'], dtype=object)

In [49]:
data.head()

,property_type,price,location,city,baths,purpose,bedrooms,Total_Area,marla,price_per_sqfeet
0,Flat,180000,7th Avenue,Islamabad,2,For Rent,3,13340,49,13.493253
1,Flat,90000,7th Avenue,Islamabad,2,For Rent,2,17424,64,5.165289
2,House,1000000,7th Avenue,Islamabad,8,For Rent,8,10890,39,91.827365
3,Room,5000,7th Avenue,Islamabad,3,For Rent,1,544,2,9.191176
4,Room,17000,7th Avenue,Islamabad,1,For Rent,2,1361,5,12.490816


In [50]:
data['purpose'].unique()

array(['For Rent', 'For Sale'], dtype=object)

In [51]:
column_name =[ 'property_type','location','city']

In [52]:
import nltk
from nltk.stem import PorterStemmer
nltk.download('punkt')
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [53]:
data['property_type'] = data['property_type'].str.lower()
data['location'] = data['location'].str.lower()
data['city'] = data['city'].str.lower()

In [54]:
data['property_type'] = data['property_type'].apply(lambda x: ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(x)]))
data['location'] = data['location'].apply(lambda x: ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(x)]))
data['city'] = data['city'].apply(lambda x: ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(x)]))

In [55]:
data.head()

,property_type,price,location,city,baths,purpose,bedrooms,Total_Area,marla,price_per_sqfeet
0,flat,180000,7th avenu,islamabad,2,For Rent,3,13340,49,13.493253
1,flat,90000,7th avenu,islamabad,2,For Rent,2,17424,64,5.165289
2,hous,1000000,7th avenu,islamabad,8,For Rent,8,10890,39,91.827365
3,room,5000,7th avenu,islamabad,3,For Rent,1,544,2,9.191176
4,room,17000,7th avenu,islamabad,1,For Rent,2,1361,5,12.490816


In [56]:
# Separate data for sale and for rent
sale_data = data[data['purpose'] == 'For Sale']
rent_data = data[data['purpose'] == 'For Rent']

In [57]:
sale_data.shape

(81475, 10)

In [58]:
rent_data.shape

(42090, 10)

In [59]:
sale_data.head()

,property_type,price,location,city,baths,purpose,bedrooms,Total_Area,marla,price_per_sqfeet
53,hous,23000000,9th avenu,islamabad,3,For Sale,4,18240,67,1260.964912
59,flat,7500000,9th avenu,islamabad,2,For Sale,2,14973,55,500.901623
62,hous,140000000,9th avenu,islamabad,3,For Sale,3,70785,259,1977.820160
63,flat,5800000,asf tower,karachi,2,For Sale,2,11434,42,507.259052
64,flat,5400000,asf tower,karachi,2,For Sale,2,12795,47,422.039859


In [60]:
features = ['property_type', 'location', 'city','baths', 'bedrooms', 'Total_Area']
target = ['price']

In [61]:
# Separate features and target for sale and rent data
X_sale = sale_data[features]
y_sale = sale_data[target]
X_rent = rent_data[features]
y_rent = rent_data[target]

In [62]:
X_rent.shape

(42090, 6)

In [63]:
X_sale.shape

(81475, 6)

In [64]:
categorical_features = ['property_type', 'location', 'city']
numeric_features = ['baths', 'bedrooms', 'Total_Area']

In [65]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# Define preprocessing steps
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

In [66]:
from sklearn.compose import ColumnTransformer
sale_preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
train_preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [67]:
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor

# Combine preprocessing and modeling steps into a pipeline
sale_pipeline = Pipeline(steps=[
    ('sale_preprocessor',sale_preprocessor),
    ('model', XGBRegressor())
])
rent_pipeline = Pipeline(steps=[
    ('train_preprocessor', train_preprocessor),
    ('model', XGBRegressor())
])

In [68]:
# Split sales data into training and testing sets
from sklearn.model_selection import train_test_split
X_sale_train, X_sale_test, y_sale_train, y_sale_test = train_test_split(X_sale, y_sale, test_size=0.3, random_state=100)

In [69]:
# Split rent data into training and testing sets
from sklearn.model_selection import train_test_split
X_rent_train, X_rent_test, y_rent_train, y_rent_test = train_test_split(X_rent, y_rent, test_size=0.3, random_state=100)

In [70]:
sale_pipeline.fit(X_sale_train,y_sale_train)

Pipeline(steps=[('sale_preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['baths', 'bedrooms',
                                                   'Total_Area']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['property_type', 'location',
                                                   'city'])])),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, ear...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [71]:
import joblib
joblib.dump(sale_pipeline, 'trained_sales_data.pkl')

['trained_sales_data.pkl']

In [72]:
y_sale_pred=sale_pipeline.predict(X_sale_test)

In [73]:
# Calculate regression metrics
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_sale_test, y_sale_pred)
r2 = r2_score(y_sale_test, y_sale_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 68530834308662.21
R-squared: 0.8728517539908308


In [74]:
rent_pipeline.fit(X_rent_train, y_rent_train)

Pipeline(steps=[('train_preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['baths', 'bedrooms',
                                                   'Total_Area']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['property_type', 'location',
                                                   'city'])])),
                ('model',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, ea...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=None, num_parallel_tree=None,
                              predictor=None, random_state=None, ...))])

In [75]:
import joblib
joblib.dump(rent_pipeline, 'trained_rent_data.pkl')

['trained_rent_data.pkl']

In [76]:
y_rent_pred=rent_pipeline.predict(X_rent_test)

In [77]:
# Calculate regression metrics
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_rent_test, y_rent_pred)
r2 = r2_score(y_rent_test, y_rent_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

Mean Squared Error: 4794277367.419014
R-squared: 0.7392254702736841


In [85]:
# Load trained models
sale_model = joblib.load('/content/trained_sales_data.pkl')
rent_model = joblib.load('/content/trained_rent_data.pkl')

In [86]:
# User interaction
choice = input("Choose one of these:  'for sale' or 'for rent': ")
selected_model = sale_model if choice == 'for sale' else rent_model

Choose one of these:  'for sale' or 'for rent': for sale


In [87]:
# Input features
property_type = input("Enter property type: ")
city = input("Enter city: ")
location = input("Enter location: ")
baths = int(input("Enter number of baths: "))
bedrooms = int(input("Enter number of bedrooms: "))
Total_Area=int(input("Enter Total Area: "))

Enter property type: house
Enter city: islamabad
Enter location: soan garden
Enter number of baths: 3
Enter number of bedrooms: 5
Enter Total Area: 1250


In [88]:
stemmed_property_type= ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(property_type)]).lower()
stemmed_city_type= ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(city)]).lower()
stemmed_location_type= ' '.join([stemmer.stem(word) for word in nltk.word_tokenize(location)]).lower()

In [89]:
# Preprocess input
input_data = pd.DataFrame({
    'property_type': [stemmed_property_type],
    'baths': [baths],
    'bedrooms': [bedrooms],
    'location': [stemmed_location_type],
    'city': [stemmed_city_type],
    'Total_Area':[Total_Area]
})

In [90]:
# Make prediction
prediction = selected_model.predict(input_data)

In [91]:
# Display prediction
print(f"Predicted house price: {prediction[0]:.2f} Rs")

Predicted house price: 7539881.00 Rs
